## 1. Introduction
The main objective of this showcase project is to build a predictive model to predict (i) loan status, approve/reject, for potential clients; (ii) credit score variable based on individual and loan-related attributes. <br>
The project uses the [Loan Approval Classification Dataset on Kaggle](https://www.kaggle.com/datasets/taweilo/loan-approval-classification-data/data?select=loan_data.csv). This report presents steps to achieve this objective. The source code of this project is available at [Github](https://github.com/ant-analytics/Beta.git).

## 2. Understand Data
Import required packages and process to load data then display the first few rows of the dataset to understand the data structure.

In [ ]:
import sys
sys.path.append('./src') # ensure the src folder in the python path
import warnings
warnings.filterwarnings('ignore')
import os
import time
import datetime

import tensorflow as tf
import keras_tuner
from keras.utils import plot_model
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

# project packages
from src.data_loader import load_data
from src.data_eda import *
from src.model_train import *
from src.hpmodel import *

# set project random state
random_state = 42

# Data loader
raw_data_path = './data/raw_loan_data.csv'
metadata_path = './data/metadata.txt'
raw_data, num_features, cat_features, metadata = load_data(raw_data_path, metadata_path)
metadata.sort_values(by='Type').reset_index(drop=True)

original_indices = raw_data.index # Keep the original indices for later use
raw_data.head(5)

Let's display some key information about features in the dataset:

In [ ]:
metadata['na_count'] = raw_data.isna().sum().values
metadata.sort_values(by='Type').reset_index(drop=True)

There are 14 features in the dataset including 5 qualitative features and 9 quantitative features. The target variable is `loan_Status` which is a qualitative feature but already coded as integer; the targer 'credit_score' is quantitative feature. This finding suggests that features encoding is required for qualitative features. Additionally, there is no missing values in the dataset <br>
Let's check key information about quantitative features in the dataset:

In [ ]:
describe_data(raw_data[num_features]).round(2).rename(columns=metadata.set_index('Feature')['Description'].to_dict()).T.sort_values(by='max', ascending=False)

As can be seen from statistic summary, the range of values for each feature is different. For example the range of income is from 80K to 7000K while the range of age is from 20 to 144. This finding sugests that the features need to be rescaled before serving as input to the model. <br>
Addtionally, there are unusual values in the dataset such as the age of a person is 144 or year of employment experiece is 125. These values are not practical and need to be treated as outliers. <br>.
Let's display a dashboard to have an over view of the dataset:

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('plots/eda_dashboard.png')

# display image
plt.imshow(img)
plt.axis('off') # hide axis
plt.show()

From the dashboard, we can see that number of reject and approve loand are imbalanced. Therefore, we should use stratified sampling methods to ensure that train, validation and test set maintain this propotion. <br>


## 3. Data Preparation
In this project, I will divide the dataset to train, validation and test set with propotion of 50-30-20, respectively. The source code for split function is available at [Github](https://github.com/ant-analytics/Beta.git).


In [ ]:
# Data split
from src.data_preprocessing import split_data, preprocess_data, transform_features
X_train, X_val, X_test, y_train_loan_status, y_val_loan_status, y_test_loan_status, y_train_score, y_val_score, y_test_score = split_data(raw_data)
print(f"Train data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")

As mentioned above, outliers with respect to person age and  are presented in the dataset. These data points should be labeled and remove from the set properly. In this project, I use Isolation Forest method to detect ouliers. Details about isolation forest can be found at [Wikipedia](https://en.wikipedia.org/wiki/Isolation_forest). To do that, I will train a outliers detector on train set, use person age, year of employment experience and length of credit history in years as features to train. 

In [ ]:
# outlier detection
from sklearn.ensemble import IsolationForest
outlier_features = ['person_age', 'person_emp_exp', 'cb_person_cred_hist_length']
outlier_detector = IsolationForest(contamination=0.009, random_state=random_state)
X_train['outliers'] = outlier_detector.fit_predict(X_train[outlier_features])
indices_to_remove = X_train[X_train['outliers'] == -1].index # Get the indices of the outliers

# Plot box plot
fig, axs = plt.subplots(3, 1, figsize=(12, 12), sharex=True)
for i, feature in enumerate(X_train[['person_age', 'person_emp_exp', 'cb_person_cred_hist_length']].columns):
    description = metadata.loc[metadata['Feature'] == feature, 'Description'].values[0]
    ax = axs[i]
    boxplot = ax.boxplot([X_train[X_train['outliers'] == 1][feature], X_train[X_train['outliers'] == -1][feature]], 
                         labels=['Inliers', 'Outliers'], vert=False)
    ax.set_title(f'Box Plot of {description}', fontweight='bold', fontsize=14)
    ax.set_xlabel(description)
    ax.set_ylabel(f'Outliers Detection')
    
    # # Add mean annotation for inliers
    # mean_inliers = boxplot['means'][0].get_xdata()[0]
    # ax.annotate(f'Mean: {mean_inliers:.2f}', xy=(mean_inliers, 1), xytext=(mean_inliers + 0.5, 1.1),
    #             horizontalalignment='center', verticalalignment='center')
    
    # # Add mean annotation for outliers
    # mean_outliers = boxplot['means'][1].get_xdata()[0]
    # ax.annotate(f'Mean: {mean_outliers:.2f}', xy=(mean_outliers, 2), xytext=(mean_outliers + 1, 2.1),
    #             horizontalalignment='center', verticalalignment='center')
    
    # Draw vertical line through min values
    # min_inliers = boxplot['whiskers'][0].get_xdata()[0]
    min_outliers = boxplot['caps'][2].get_xdata()[0]
    # ax.axvline(min_inliers, color='r', linestyle='--', label='Min Inliers')
    ax.axvline(min_outliers, color='r', linestyle='--', label='Min Outliers')
    
    # Add tick label for red line
    # ax.annotate(f'{min_inliers:.2f}', xy=(min_inliers, 0.5), xytext=(min_inliers, 0.5),
    #             color='red', horizontalalignment='center', verticalalignment='center')
    # ax.annotate(f'{min_outliers:.2f}', xy=(min_outliers, 1.5), xytext=(min_outliers, 1.5),
    #             color='red', horizontalalignment='center', verticalalignment='center')
    
    ax.legend()

fig.tight_layout()
plt.show()


As we can see from the plot, the outlier dectector are reasonable to detect outlier in the dataset. We will use this detector to remove outliers from the validation and test set.

In [7]:
# Remove outliers
X_train.drop(index=indices_to_remove, columns='outliers', inplace=True)
y_train_loan_status.drop(labels=indices_to_remove, inplace=True)
y_train_score.drop(labels=indices_to_remove, inplace=True)

# Remove outliers from validation set
X_val['outliers'] = outlier_detector.predict(X_val[outlier_features])
val_indices_to_remove = X_val[X_val['outliers'] == -1].index
X_val.drop(index=val_indices_to_remove, columns='outliers', inplace=True)
y_val_loan_status.drop(labels=val_indices_to_remove, inplace=True)
y_val_score.drop(labels=val_indices_to_remove, inplace=True)

# Remove outliers from test set
X_test['outliers'] = outlier_detector.predict(X_test[outlier_features])
test_indices_to_remove = X_test[X_test['outliers'] == -1].index
X_test.drop(index=test_indices_to_remove, columns='outliers', inplace=True)
y_test_loan_status.drop(labels=test_indices_to_remove, inplace=True)
y_test_score.drop(labels=test_indices_to_remove, inplace=True)

The next step is to preprocess data. Two tasks need to be done: (i) encode categorical variables and (ii) scale numerical variables. For the first task, I will use ordinal endcode method to encode categorical variables. For the second task, I will use standard scaler to scale numerical variables. The source code for these tasks is available at [Github](https://github.com/ant-analytics/Beta.git).

In [8]:
# Data preprocessing
X_train_transform, X_val_transform, X_test_transform, y_train_score_transform, y_val_score_transform, y_test_score_transform, col_transformer, y_scaler = preprocess_data(
    X_train, X_val, X_test, y_train_score, y_val_score, y_test_score, num_features, cat_features)

# keep track features for interpretate
transform_features = transform_features(col_transformer)

In [ ]:
#  plot features after transform
fig, axs = plt.subplots(4, 3, figsize=(24, 18), sharex=True)
for i, feature in enumerate(transform_features):
    ax = axs[i//3, i%3]
    boxplot = ax.boxplot(X_test_transform[:, i], showfliers=True, vert=False, showmeans=True, meanline=True)
    description = metadata.loc[metadata['Feature'] == feature, 'Description'].values[0]
    ax.set_title(f'Box Plot of {description}', fontweight='bold', fontsize=14)
    ax.yaxis.set_ticklabels([])  # Set y-tick labels off
    
    # Add mean annotation
    # mean = boxplot['means'][0].get_xdata()[0]
    # ax.annotate(f'Mean: {mean:.2f}', xy=(mean, 1), xytext=(mean, 1.1),
    #             arrowprops=dict(facecolor='black', shrink=0.05),
    #             horizontalalignment='center', verticalalignment='center')
    
    # Add x-axis label
    ax.set_xlabel('Value')
    
    # Add legend for mean and median lines
    ax.legend([boxplot['means'][0], boxplot['medians'][0]], ['Mean', 'Median'], loc='upper right')

fig.tight_layout()
plt.show()

## 4. Model Development and Training
The purpose of this project is to predict loan status and credit score. These outputs are different in nature. Loan status is a binary output coded as 1 for approve and 0 for reject. Meanwhile, credit score is a continuous variable. Thus, our problems are classification and regression problems. The predictive models use the same set of input data. A popular choice for this type of problem is to develop two separate models for each problem. However, in this project, I will use a single neural network model to predict both loan status and credit score.<br>
The next step is to build the model architecture, set up hyperparameters, and train the network. A common approach to developing the network is to start with a reasonable architecture and hyperparameters, then tune them to improve the model performance. But, in this project, I will do it in reverse order. I will start with tuning hyperparameters, then choose the "best" model architecture. From this point, I will train the model and evaluate its performance.

In [ ]:
# Hyperparameter tuning
tuner_log_dir = f"logs/tuner/tuner_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
tuner_save_check_point = f'save_tuner/checkpoint_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras'

# Create TensorBoard callback
tuner_tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=tuner_log_dir,  
    histogram_freq=1,
    profile_batch='500,520'
)

tuner_early_stop_callback = keras.callbacks.EarlyStopping(
    monitor='val_output_status_accuracy',
    patience=3,
    mode='max'
)

tuner_check_point = keras.callbacks.ModelCheckpoint(
    filepath='best_tuner.keras',
    monitor='val_output_status_accuracy',
    save_best_only=True,
)

tuner_callbacks = [tuner_tensorboard_callback, tuner_early_stop_callback, tuner_check_point]
 
tuner = keras_tuner.GridSearch(
    hypermodel=build_hyper_model,
    objective=[
        keras_tuner.Objective('val_output_status_accuracy', direction='max'),
        keras_tuner.Objective('val_output_score_mean_squared_error', direction='min')
    ],
    max_trials=50,
    directory=tuner_log_dir,
    project_name='loan_default_prediction',
    overwrite=True
)
# Tuning hyperparameters
tuner.search(
    X_train_transform, y=[y_train_loan_status, y_train_score_transform],
    validation_data=(X_val_transform, [y_val_loan_status, y_val_score_transform]), epochs=50,
    callbacks=tuner_callbacks
)

# Retrieve and save the top 10 best models
save_dir = 'save_tuner/top_best/'
os.makedirs(save_dir, exist_ok=True)
best_models = tuner.get_best_models(num_models=10)
for i, model in enumerate(best_models):
    model.save(f'save_tuner/top_best/{i+1}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

save_worst = 'save_tuners/top_worst/'
os.makedirs(save_worst, exist_ok=True)
worst_models = tuner.get_best_models(num_models=-10)
for i, model in enumerate(worst_models):
    model.save(f'save_models/top_worst/{i+1}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras')

After tuning, choose the best model then plot model architecture.

In [ ]:
# load the best model
best_model = tuner.get_best_models(num_models=1)[0]
plot_model(best_model, to_file='plots/best_model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True, show_trainable=True, show_dtype=True, dpi=96)


Now, it is time to train the model. The training process will take a while. After training, we will evaluate the model performance on the validation set. The evaluation metrics for classification and regression problems are accuracy and mean squared error, respectively.

In [ ]:
#  calbacks for fit model
fit_log_dir = f"logs/fits/fit_{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}"
fit_save_check_point = f'save_fits/checkpoint_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.keras'

# Create TensorBoard callback
fit_tensorboard_callback = keras.callbacks.TensorBoard(
    log_dir=fit_log_dir,  
    histogram_freq=1,
    profile_batch='500,520'
)

fit_early_stop_callback = keras.callbacks.EarlyStopping(
    monitor='val_output_status_accuracy',
    patience=3,
    mode='max'
)

fit_check_point = keras.callbacks.ModelCheckpoint(
    filepath=fit_save_check_point,
    monitor='val_output_status_accuracy',
    save_best_only=True,
)

fit_callbacks = [fit_tensorboard_callback, fit_early_stop_callback, fit_check_point]

#  fit the best model
history = best_model.fit(
    X_train_transform, [y_train_loan_status, y_train_score_transform],
    validation_data=(X_val_transform, [y_val_loan_status, y_val_score_transform]),
    epochs=100, callbacks=fit_callbacks, shuffle=True
)

# model evaluation
evaluation = best_model.evaluate(X_test_transform, [y_test_loan_status, y_test_score_transform], return_dict=True)

## 5. Conclusion and Discussion 

This project has built a a predictive model to predict loan status and credit score base of 45K sample dataset. The model use IsolationForest to detect outliers and use a single neural network model to predict both loan status and credit score. However, this project stil need improvement in the following aspects: (i) a multi criteria should be use to monitor hypertuning process; (ii) should develop a different models to comparison./. <br>